In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import ast
import numpy as np


def fun_1(train_overall,classnumber=99):
    if classnumber==99:
        max_value = list(map(max, zip(train_overall[0], train_overall[1],train_overall[2])))
        min_value = list(map(min, zip(train_overall[0], train_overall[1],train_overall[2])))
        avg_value = list(map(sum, zip(train_overall[0], train_overall[1],train_overall[2])))
        avg_value = [x/3 for x in avg_value]
    else:
        max_value = list(map(max, zip(train_overall[0][classnumber], train_overall[1][classnumber],train_overall[2][classnumber])))
        min_value = list(map(min, zip(train_overall[0][classnumber], train_overall[1][classnumber],train_overall[2][classnumber])))
        avg_value = list(map(sum, zip(train_overall[0][classnumber], train_overall[1][classnumber],train_overall[2][classnumber])))
        avg_value = [x/3 for x in avg_value]
    return max_value,min_value,avg_value

/home/simrank/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def path_network_fn(type_network,ds):
    if type_network=='MLP':
        if ds=='MNIST':
            network_path = '/mnt/NVME2TB/simran/PCA/MLP_MNIST/Network'

        if ds=='CIFAR10':
            network_path = '/mnt/NVME2TB/simran/PCA/MLP_CIFAR10/Network'
            
    if type_network =='CNN':
        if ds=='FashionMNIST':
            network_path = '/mnt/NVME2TB/simran/PCA/FashionMNIST/Network'
            
        if ds=='MNIST':
            network_path = '/mnt/NVME2TB/simran/PCA/MNIST/Network'
            
        if ds=='CIFAR10':
            network_path = '/mnt/8TB/simran/PCA/CIFAR_10_Wodrop/Network'
            
    if type_network=='AlexNet':
        if ds=='TinyImageNet':
            network_path = '/mnt/NVME2TB/simran/PCA/TinyImagenet_Alexnet/Network'
        if ds=='CIFAR100':
            network_path = '/mnt/8TB/simran/PCA/CIFAR_100/Network'
    if type_network=='ResNet18':
        if ds=='CIFAR10':
            network_path = f'../Modern_backbones/models/{ds}_{type_network}/Network'
        
    return network_path

def epoch_number(network_path,corrupt,run):
    models=sorted(os.listdir(f'{network_path}/{corrupt}/Run_{run}'))
    model_numbers = [
        (m, int(m.split("_")[1].split(".")[0]))  
        for m in models if m.startswith("model_") and m.split("_")[1].split(".")[0].isdigit()
    ]
    model_highest = max(model_numbers, key=lambda x: x[1])
    return model_highest[1]

In [3]:
def path_results_resnet(ds,network,corrupt,run,epoch):
    net_corr=f'../Modern_backbones/results/40/angle_results_0.99/{ds}_{network}/results_{corrupt}/angle_results/Run_{run}/{epoch}'
    net_org=f'../Modern_backbones/results/40/angle_results_0.99/{ds}_{network}/results_{corrupt}/angle_results_exp3/Run_{run}/{epoch}'
    net_mem=f'../Modern_backbones/results/40/angle_results_0.99/{ds}_{network}/results_uncorrupted/results_{corrupt}/angle_results/Run_{run}/{epoch}'
    return net_corr,net_org,net_mem

def path_results(ds,network,corrupt,run):
    if network=='MLP':
        net_corr=f'../../pca_angle/angle_results/{ds}_{network}/SGD/results_{corrupt}/angle_results/Run{run}/cor_label'
        net_org=f'../../pca_angle/angle_results/{ds}_{network}/SGD/results_{corrupt}/angle_results_exp3/Run{run}/cor_label'
        net_mem=f'../../pca_angle/angle_results/{ds}_{network}/SGD/results_uncorrupted/results_{corrupt}/angle_results/Run{run}/cor_label'
        
    elif network=='CNN':
        net_corr=f'../../pca_angle/angle_results/{ds}_{network}/SUBSPACE/results_{corrupt}/angle_results/Run_{run}/cor_label'
        net_org=f'../../pca_angle/angle_results/{ds}_{network}/SUBSPACE/results_{corrupt}/angle_results_exp3/Run_{run}/cor_label'
        net_mem=f'../../pca_angle/angle_results/{ds}_{network}/SUBSPACE/results_uncorrupted/results_{corrupt}/angle_results/Run_{run}/cor_label'
    elif network=='AlexNet'and ds=='TinyImageNet':
        net_corr=f'../../pca_angle/angle_results/{ds}_{network}/results/results_{corrupt}/angle_results/Run_{run}/cor_label'
        net_org=f'../../pca_angle/angle_results/{ds}_{network}/results/results_{corrupt}/angle_results_exp3/Run_{run}/cor_label'
        net_mem=f'../../pca_angle/angle_results/{ds}_{network}/results/results_uncorrupted/results_{corrupt}/angle_results/Run_{run}/cor_label'
    elif network=='AlexNet'and ds=='CIFAR100':
        net_corr=f'../../pca_angle/angle_results/{ds}_{network}/results_{corrupt}/angle_results/Run_{run}/cor_label'
        net_org=f'../../pca_angle/angle_results/{ds}_{network}/results_{corrupt}/angle_results_exp3/Run_{run}/cor_label'
        net_mem=f'../../pca_angle/angle_results/{ds}_{network}/results_uncorrupted/results_{corrupt}/angle_results/Run_{run}/cor_label'
    elif network=='ResNet18'and ds=='CIFAR10':
        network_path=path_network_fn(network,ds)
        epoch=epoch_number(network_path,corrupt,run)
        net_corr,net_org,net_mem=path_results_resnet(ds,network,corrupt,run,epoch)
    return net_corr,net_org,net_mem

def path_results_exp3_cnn(ds,network,corrupt,run):
    if ds=='MNIST' or ds=='FashionMNIST':
        net_path=f'../../pca_angle/angle_results/{ds}_{network}/SUBSPACE/results_uncorrupted/results_{corrupt}/angle_results/{run}/cor_label'
    if ds=='CIFAR10':
        net_path=f'../../pca_angle/angle_results/{ds}_{network}/SUBSPACE/results_uncorrupted/results_{corrupt}/angle_results/Run_{run}/cor_label'
    return net_path
def path_project(ds,network,corrupt,run,subspace_type):
    if subspace_type=='corrupt':
        net_path,_,_=path_results(ds,network,corrupt,run)
    elif subspace_type=='original':
        _,net_path,_=path_results(ds,network,corrupt,run)
    elif subspace_type=='exp3':
        if network=='CNN':
            net_path=path_results_exp3_cnn(ds,network,corrupt,run)
        else:
            _,_,net_path=path_results(ds,network,corrupt,run)
    return net_path

In [10]:
def best_layer_val(ds,network,corrupt,subspace_type):
    runs=4
    if network=='MLP':
        test_overall=[]
        for run in range(1,runs):
            net_path=path_project(ds,network,corrupt,run,subspace_type)
            test_overall.append(pd.read_csv(f'{net_path}/acc_overall_test.csv')['acc_overall'].tolist())
        test_overall=np.array(test_overall)
        _,_,avg_value_test=fun_1(test_overall)
        best_layer_value =np.max(avg_value_test[::2][:-1])

    elif network=='CNN':
        data_layer_name=['input','flattern','fc1','fc2','fc3']
        test_overall=[]
        for run in range(1,runs):
            data_test=[]
            for data_layer in data_layer_name:
                net_path=path_project(ds,network,corrupt,run,subspace_type)
                data_test.append(pd.read_csv(f'{net_path}/layer_{data_layer}_acc_overall_test.csv')['acc_overall'][0])
            test_overall.append(data_test)
        _,_,avg_value_test=fun_1(test_overall)    
        best_layer_value =np.max(avg_value_test)

    elif network=='AlexNet':
        data_layer_name=['flattern','fc1','fc2']
        test_overall=[]
        for run in range(1,runs):
            data_test=[]
            for data_layer in data_layer_name:
                net_path=path_project(ds,network,corrupt,run,subspace_type)
                data_test.append(pd.read_csv(f'{net_path}/layer_{data_layer}_acc_overall_test.csv')['acc_overall'][0])
            test_overall.append(data_test)
        _,_,avg_value_test=fun_1(test_overall)    
        best_layer_value =np.max(avg_value_test)
    elif network=='ResNet18':
        data_layer_name=['l0','l0_1','l0_2','l0_3','l1','l2','l3','l4','bf_last']#
        test_overall=[]
        for run in range(1,runs):
            data_test=[]
            for data_layer in data_layer_name:
                net_path=path_project(ds,network,corrupt,run,subspace_type)
                data_test.append(pd.read_csv(f'{net_path}/layer_{data_layer}_acc_overall_test.csv')['acc_overall'][0])
            test_overall.append(data_test)
        _,_,avg_value_test=fun_1(test_overall)    
        best_layer_value =np.max(avg_value_test)
    return best_layer_value

In [11]:
models=['MLP','MLP','CNN','CNN','CNN','AlexNet','AlexNet','ResNet18']
datasets=['MNIST','CIFAR10','MNIST','FashionMNIST','CIFAR10','CIFAR100','TinyImageNet','CIFAR10']
corrupt_prob=[0.2,0.4,0.6,0.8]

In [12]:
subspace_types=['corrupt','original']#
for subspace_type in subspace_types:
    print(f"MASC {subspace_type}  | {corrupt_prob[0]}| {corrupt_prob[1]}| {corrupt_prob[2]}| {corrupt_prob[3]}")
    for network,ds in zip(models,datasets):
        print(f"{network}-{ds}","|",best_layer_val(ds,network,corrupt_prob[0],subspace_type),"|",
             best_layer_val(ds,network,corrupt_prob[1],subspace_type),"|",
              best_layer_val(ds,network,corrupt_prob[2],subspace_type),"|",
             best_layer_val(ds,network,corrupt_prob[3],subspace_type))

MASC corrupt  | 0.2| 0.4| 0.6| 0.8
MLP-MNIST | 0.9693 | 0.9646 | 0.9481333333333334 | 0.8343000000000002
MLP-CIFAR10 | 0.5343666666666667 | 0.5020666666666667 | 0.449 | 0.3243333333333333
CNN-MNIST | 0.9610333333333334 | 0.9515666666666668 | 0.9359666666666667 | 0.8521000000000001
CNN-FashionMNIST | 0.8356333333333333 | 0.819 | 0.7999999999999999 | 0.7390666666666666
CNN-CIFAR10 | 0.6187 | 0.4904333333333333 | 0.3933333333333333 | 0.2934333333333334
AlexNet-CIFAR100 | 0.37989999999999996 | 0.3143666666666666 | 0.15903333333333333 | 0.04973333333333333
AlexNet-TinyImageNet | 0.12426666666666668 | 0.08363333333333334 | 0.029366666666666666 | 0.008333333333333333
ResNet18-CIFAR10 | 0.6711666666666667 | 0.6201333333333333 | 0.5767666666666668 | 0.4387666666666667
MASC original  | 0.2| 0.4| 0.6| 0.8
MLP-MNIST | 0.9695333333333335 | 0.9673666666666666 | 0.9629333333333333 | 0.9578666666666665
MLP-CIFAR10 | 0.5397 | 0.5256666666666666 | 0.5099333333333332 | 0.48446666666666666
CNN-MNIST | 0.9

# model accuracy

In [95]:
def path_acc(dataset,type_network,corrupt):
    accu_path=f'../model_accuracy/{dataset}_{type_network}/{corrupt}'
    if type_network=='ResNet18':
        accu_path=f'../Modern_backbones/models/{dataset}_{type_network}/Accuracy_results/{corrupt}'
    return accu_path

In [114]:
def model_accuracy(ds,type_network,corrupt):
    
    accuracy_path=path_acc(ds,type_network,corrupt)
    runs=4
    test_model=[]
    
    for run in range(1,runs):
        if type_network=='MLP':
            columnnumber=-2
            file_name=f'Run{run}_Accuracy.csv'
            test_model.append(pd.read_csv(f'{accuracy_path}/{file_name}')['Test_accuracy'].tolist()[columnnumber])

        else:
            columnnumber=-1
            file_name=f'Run_{run}.csv'
            test_model.append(pd.read_csv(f'{accuracy_path}/{file_name}')['Test_accuracy'].tolist()[columnnumber]/100)

    avg_model =np.average(test_model)
    return avg_model


In [115]:
models=['MLP','MLP','CNN','CNN','CNN','AlexNet','AlexNet','ResNet18']
datasets=['MNIST','CIFAR10','MNIST','FashionMNIST','CIFAR10','CIFAR100','TinyImageNet','CIFAR10']
corrupt_prob=[0.2,0.4,0.6,0.8]

In [116]:
print(f"Model accuracy | {corrupt_prob[0]}| {corrupt_prob[1]}| {corrupt_prob[2]}| {corrupt_prob[3]}")

for network,ds in zip(models,datasets):
    print(f'{network}-{ds}  |  {model_accuracy(ds,network,corrupt_prob[0])} | {model_accuracy(ds,network,corrupt_prob[1])} | {model_accuracy(ds,network,corrupt_prob[2])} | {model_accuracy(ds,network,corrupt_prob[3])}')
    

Model accuracy | 0.2| 0.4| 0.6| 0.8
MLP-MNIST  |  0.8738000194231669 | 0.7328000068664551 | 0.5416666666666666 | 0.3209666609764099
MLP-CIFAR10  |  0.48623334368069965 | 0.4035333395004272 | 0.30550000071525574 | 0.1968666662772496
CNN-MNIST  |  0.8751666666666668 | 0.6943999999999999 | 0.4710666666666667 | 0.2830333333333333
CNN-FashionMNIST  |  0.7774333333333333 | 0.6135 | 0.43260000000000004 | 0.25573333333333337
CNN-CIFAR10  |  0.6048333333333334 | 0.4615333333333333 | 0.30963333333333337 | 0.18320000000000003
AlexNet-CIFAR100  |  0.2844 | 0.20533333333333334 | 0.09646666666666666 | 0.034300000000000004
AlexNet-TinyImageNet  |  0.09746666666666666 | 0.05449999999999999 | 0.0202 | 0.007300000000000001
ResNet18-CIFAR10  |  0.6739333333333333 | 0.5153333333333334 | 0.34480000000000005 | 0.20020000000000002
